# <span style = "color:black">**Problem Setting**

Bayesian Neural Network는 **parameter와 output의 분포그 자체를 구해내고자 하는 것이 목적**이다. 분포를 구한다면 **모델의 예측이 얼마나 믿을만한지,얼마나 불확실한지를 알 수 있다는 장점이 있기 때문**이다.

![](./images/standard%20vs%20bayesian.PNG)

이 과정에서 파라미터에 대한 posterior(사후확률분포)를 구하게 되는데 문제는 여기서 발생한다.

\begin{aligned}
p(z|x) = \frac{p(x|{z})p(x)}{p(x)} = \frac{p(x|z)p(x)}{\int p(x|z)p(x)dz}
\end{aligned} 

- $z$는 latent variable로 숨겨져 있으며(이러한 의미에서 hidden variable이라고도 한다.) 관측가능한 변수인 $x$의 값에 영향을 미치는 확률변수(random variable)이다. observed data로부터 계산을 통해서만 구할 수 있으며 Bayesian Nueral Network에서는 $w$이다.

분자의 likelyhood와 prior는 MAP나 ML에서와 마찬가지로 우리가 알고있는 비교적 계산이 편리한 분포로 주로 가정한다. 하지만 분모는 수많은 weight에 대한 다중적분이기 때문에 계산하기가 매우 어렵다. 따라서 **posterior를 근사적으로 구할 수 있는 방법들이 여러가지가 연구되었으며 Variational Inference는 이 중 한가지 방법**이다.

이번 포스트에서는 어떻게 AutoEncoder가 Variational Inference를 통해서 어떻게 **posterior를 근사할 수 있는지 알아보며 또한 (Probabilistic) Generative model로서 어떻게 작동하는지** 살펴본다.

# <span style = "color:black">**Method**

## <span style = "color:black">**Latent Variable Model Assumption**

Variational Auto-Encoder는 Gradient를 기반으로 학습하는 latent variable model이다. latent variable model에서는 관측된 값 $x$가 latent $z$로부터 generated 되었다고 가정한다. latent variable들은 관측된 값의 structure에 대한 정보를 담고있으며 바로 눈에 보이지 않는 숨겨진 변수이다. (예시 추가할지말지 고민 ...)

$$z \rightarrow x$$

뿐만 아니라 두 확률변수에 대한 joint distribution이 존재하며 parameter $\theta$가 있다고 가정한다. 이는 joint distribution으로부터 얻을 수 있는  conditional distribution,marginal distribution 또한 모두 $\theta$로 parameterize 됨을 의미한다.

$$p_{\theta}(z,x)$$

## <span style = "color:black">**Variational Inference,후에 더 적절한 제목 찾기**

Variational Inference에서는 어떻게 posetrior를 얻을 수 있을까? **우리는 $z$에 관한 또다른 확률분포 $q_{\phi}(z|x)$를 가정한 후 이를 점점 실제 posterior인 $p_{\theta}(z|x)$에 다가가도록 한다.** 이때 $q_{\phi}$는 우리가 알고있는 비교적 계산이 편리한 함수로 가정한다.(예를 들면 정규분포가 있다.) 따라서 얼마나 다가가야하는지 알기 위해서는 먼저 두 분포가 얼마나 "차이"나는지 알아야 하며 대표적인 값인 KL-divergence를 사용한다.

\begin{aligned}
\text{D}_{KL}[q_{\phi}(z|x)||p_{\theta}(z|x)] &:= \int_zq_{\phi}(z|x)\text{log}\frac{q_{\phi}(z|x)}{p_{\theta}(z|x)}dz \\
\end{aligned}

그렇다면 우리의 목적은 다음과 같이 KL-divergence를 가장 줄이는 $\hat{q_{\phi}}$를 찾는 문제와 같다.

$${\hat{q_{\phi}}} = \underset{q_{\phi}}{\text{argmin}}\,\text{D}_{KL}[q_{\phi}(z|x)||p_{\theta}(z|x)] $$

여기서 RHS term인 KL-divergence는 ELBO와 evidence(marginal log likelyhood)의 차이로 전개할 수 있다.

\begin{aligned}
&
\begin{aligned}
\text{D}_{KL}[q_{\phi}(z|x)||p_{\theta}(z|x)] &:= \int_zq_{\phi}(z|x)\text{log}\frac{q_{\phi}(z|x)}{p_{\theta}(z|x)}dz \\
&= -\mathcal{L}(\phi,\theta;x) + \text{log}\,p_{\theta}(x)\\
\end{aligned}
\\
&\text{where } \mathcal{L}(\phi,\theta;x) := \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{\theta}(x,z)}{q_{\phi}(z|x)} \right] = \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{\theta}(x,z)-\text{log}\,q_{\phi}(z|x) \right]
\end{aligned}



- evidence를 $\phi,\theta$의 함수로 notation한 이유는 이후의 optimization에서는 변수이기.(MLE에서 parameter는 하나의 고정된 상수이지만 일단은 변수로 보고 likelyhood의 max값을 찾는 과정과 비슷한 느낌인 것 같다.)
- 관측값$x$는 고정이므로 ;뒤에 놓았다.(이것도 MLE에서 관측한 값은 고정인 느낌과 같다.) 우리의 목적은 관측된 값 $x$가 있을 때의 latent variable $z$의 posterior를 구하는 것이다.

여기서 $p_{\theta}(x)$는 $q_{\phi}$와 독립적이므로 $q_{\phi}$를 maximization하는 과정에서는 상수취급 할 수 있으므로 이는 ELBO를 maximization하는 문제로 바뀌게 된다.

\begin{aligned}
\hat{q_{\phi}} &= \underset{q_{\phi}}{\text{argmax}}\,\mathcal{L}(\phi,\theta;x) \\
&= \underset{q_{\phi}}{\text{argmax}}\,\mathbb{E}_{Z\sim q}\left[\text{log}\,p_{\theta}(x,z)-\text{log}\,q_{\phi}(z|x) \right]
\end{aligned}

또한 $z$는 integral후에 사라지는 적분변수이며 $x$는 관측값이다. 결국 $q_{\phi}$를 control하여 KL-divergence를 maximize하는 것은 $\phi$를 control하여 KL-divergence를 maximize하는 $\hat{\phi}$를 찾는 것과 같은 문제이다.

\begin{aligned}
\hat{\phi} = \underset{\phi}{\text{argmax}}\,\mathcal{L}(\phi;\theta,x)
\end{aligned}

- maximization 과정에서는 $\phi$만 움직이는 변수 취급하므로 $\theta$는 일단 고정된 값으로 취급하였다.

정리하자면 문제는 원래 우리가 임의로 설정한 posterior인 $q_{\phi}$를 true posterior인 $p(z|x)$와 비슷하게 만드는 문제였다. 이는 KL-divergence를 minimize하는 $q_{\phi}$를 찾는 것이었으며 수식전개를 통해서 결국에는 ELBO를 maximize하는 $\phi$를 찾는 문제와 같음을 알 수 있다.

## <span style = "color:black">**generalization model**

# <span style = "color:black">**Appendix**

# <span style = "color:black">**evidence lower bound(ELBO)**

먼저 evidence는 다음과 같이 marginzalization된 loglikelyhood로 정의한다.

$$\text{evidence} := \text{log}\,p(x;\theta) = \int_z p_{\theta}(x,z)dz$$

evidence를 쭉 전개하면 다음과 같다.

\begin{aligned}
\text{log}\,p(x;\theta) &= \int_z p_{\theta}(x,z)dz \\
&=\text{log}\,\int_z \frac{p_{\theta}(x,z)}{q_{\phi}(z|x)}q_{\phi}(z|x)dz \\
&=\text{log}\,\mathbb{E}_{Z\sim q}\left[\frac{p_{\theta}(x,z)}{q_{\phi}(z|x)} \right]
\end{aligned}

[Jenson's inequality](https://ko.wikipedia.org/wiki/%EC%98%8C%EC%84%BC_%EB%B6%80%EB%93%B1%EC%8B%9D) $f(E[X])\leq E[f(X)]$에 의하여 evidence의 lowerbound를 찾을 수 있다. 이를 evidence lower bound(ELBO)라 정의한다.

\begin{aligned}
&\text{log}\,p(x;\theta) \geq \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{\theta}(x,z)}{q_{\phi}(z|x)} \right]\\
&\mathcal{L(\phi,\theta;x)} := \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{\theta}(x,z)}{q_{\phi}(z|x)} \right] = \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{\theta}(x,z)-\text{log}\,q_{\phi}(z|x) \right]
\end{aligned}

- evidence를 $\phi,\theta$의 함수로 notation한 이유는 optimization 과정에서 변수로 보고 사용하기 때문이다.(MLE에서 parameter는 하나의 고정된 상수이지만 일단은 변수로 보고 likelyhood의 max값을 찾는 과정과 비슷한 느낌인 것 같다.)
- 관측값$x$는 고정이므로 ;뒤에 놓았다.(이것도 MLE에서 관측한 값은 고정인 느낌과 같다.)

# <span style = "color:black">**KL-divergence 전개**

\begin{aligned}
\text{D}_{KL}[q_{\phi}(z|x)||p_{\theta}(z|x)] &:= \int_zq_{\phi}(z|x)\text{log}\frac{q_{\phi}(z|x)}{p_{\theta}(z|x)}dz \\
&= \int_zq_{\phi}(z|x)\text{log}\frac{q_{\phi}(z|x)p_{\theta}(x)}{p_{\theta}(x,z)}dz \\
&= \int_zq_{\phi}(z|x)\text{log}\frac{q_{\phi}(z|x)}{p_{\theta}(x,z)}dz  + \int_z q_{\phi}(z|x)\text{log}\,p_\theta(x)dz\\
&= -\mathcal{L}(\phi,\theta;x) + \text{log}\,p_{\theta}(x)
\end{aligned}